In [2]:
import gzip
import json
from pathlib import Path
import rdflib
import subprocess

In [3]:
in_file_1 = '/mnt/dane/work/rdf_streaming/sripas_weather/weather.json.gz'
in_file_2 = '/mnt/dane/work/rdf_streaming/sripas_weather/data.json.gz'
out_dir = Path('../data/out/sripas-weather-graphs')

In [10]:
num = 0
dir_levels = 1

def do_file(buf: str):
    global num
    filename = f'{num:010}'
    item_dir = out_dir
    for d in range(-dir_levels, 0):
        item_dir = Path(item_dir, filename[3 * (d - 1) : 3 * d])
    item_dir.mkdir(parents=True, exist_ok=True)

    with open('temp/data.json', 'wt') as fp:
        fp.write(buf)

    s_res = subprocess.run([
        'java', '-jar', 'bin/mapper.jar',
        '-m', 'rml/sripas_weather_graphs.ttl',
        '-o', str(Path(item_dir, filename + '.trig')),
        '-s', 'trig',
    ])
    if s_res.returncode != 0:
        print(s_res.stderr.decode('utf-8'))
        return
    if num % 10000 == 0:
        print(num)
    num += 1

with gzip.open(in_file_1, 'rt') as f:
    for line in f:
        do_file(line)

with gzip.open(in_file_2, 'rt') as f:
    buffer = ''
    for line in f:
        buffer += line
        if line.startswith('}'):
            data = json.loads(buffer)['payload']
            do_file(json.dumps(data))
            buffer = ''


0


KeyboardInterrupt: 